In [6]:
import folium
import pandas as pd

# Function to create the map
def create_interactive_map(csv_file, output_map_file='interactive_map.html'):
    try:
        # Load the CSV file into a DataFrame
        data = pd.read_csv('Project data2_out.csv')

        # Print the column names for debugging
        print(f"Columns in CSV: {data.columns.tolist()}")

        # Required columns
        required_columns = ['latitude', 'longitude', 'name', 'location', 'type']

        # Check if all required columns exist (case insensitive)
        data.columns = data.columns.str.strip()  # Strip any leading/trailing spaces
        missing_columns = [col for col in required_columns if col not in data.columns]

        if missing_columns:
            raise ValueError(f"CSV file is missing the following required columns: {', '.join(missing_columns)}")

        # Create a base map centered around the average of all provided coordinates
        start_coords = (data['latitude'].mean(), data['longitude'].mean())
        interactive_map = folium.Map(location=start_coords, zoom_start=10)

        # Add tile layers for road networks and satellite view
        folium.TileLayer('openstreetmap', name='Road Map').add_to(interactive_map)  # Default road network
        folium.TileLayer('Stamen Toner', name='Stamen Toner').add_to(interactive_map)  # Black and white road map
        folium.TileLayer('Stamen Terrain', name='Terrain Map').add_to(interactive_map)  # Terrain map
        folium.TileLayer(
            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr='Esri',
            name='Satellite',
            overlay=False,
            control=True
        ).add_to(interactive_map)  # Satellite view

        # Add Layer control to toggle between different map views
        folium.LayerControl().add_to(interactive_map)

        # Plot each data point with a label
        for _, row in data.iterrows():
            folium.Marker(
                location=(row['latitude'], row['longitude']),
                popup=f"Name: {row['name']}<br>Location: {row['location']}<br>Type: {row['type']}",
                icon=folium.Icon(color='blue', icon='info-sign')
            ).add_to(interactive_map)
            
            # Add a permanent label for the name
            folium.Marker(
                location=(row['latitude'], row['longitude']),
                icon=folium.DivIcon(
                    html=f"""<div style="font-size: 12pt; color : black">{row['name']}</div>"""
                )
            ).add_to(interactive_map)

        # Save the map to an HTML file
        interactive_map.save(output_map_file)
        print(f"Map has been created and saved as {output_map_file}")

    except FileNotFoundError:
        print(f"Error: The file {csv_file} was not found.")
    except pd.errors.EmptyDataError:
        print("Error: The CSV file is empty.")
    except ValueError as ve:
        print(f"ValueError: {ve}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Example usage
if __name__ == "__main__":
    csv_file_path = 'Project data2_out.csv'  # Ensure this is the correct path to your CSV file
    create_interactive_map(csv_file_path)


Columns in CSV: ['latitude', 'longitude', 'name', 'location', 'type']
Map has been created and saved as interactive_map.html
